In [3]:
#####################################
## modules et bibliothèques python ##
#####################################
# Import library
import numpy as np
import xarray as xr
from netCDF4 import Dataset

from matplotlib import pyplot as plt
import matplotlib.dates as mpd
from mpl_toolkits.basemap import Basemap
#import cartopy
#import cartopy.crs as ccrs

import os
import sys
import random

# ORCA025 GRID

In [4]:
###########
## TMASK ##
###########
#################################
## Repertoire et fichiers clés ##
#################################

dirsrc = '/gpfswork/rech/hjl/uab91nb/' 
dirdat = dirsrc + "Script/"
print (os.path.exists(dirdat))
nfdata = dirdat + "eORCA025.L75_domain_cfg_closed_seas_greenland.nc"
print (os.path.isfile(nfdata))
#ncdat = xr.open_dataset(nfdata,"r")
ncdat  = Dataset(nfdata,"r")
print ("Meshmask file is opened")
print (ncdat.data_model)
#
# read temperature field from thetao file
tmasksurf = ncdat.variables["mppmask"][0,:,:]
e1t = ncdat.variables["e1t"][0,:,:]
e2t = ncdat.variables["e2t"][0,:,:]
ny,nx = tmasksurf.shape
print(ny,nx)
#
ncdat.close()

True
True
Meshmask file is opened
NETCDF4


<ipython-input-4-1dd694c4ff79>:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tmasksurf = ncdat.variables["mppmask"][0,:,:]


1207 1442


<ipython-input-4-1dd694c4ff79>:20: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  e1t = ncdat.variables["e1t"][0,:,:]
<ipython-input-4-1dd694c4ff79>:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  e2t = ncdat.variables["e2t"][0,:,:]


## Initialisation: 1 particle per cell

In [ ]:
# Create initial_positions.txt file and fill it
#fnc = dirdat + "/initial_positions_bottom_ORCA12.txt"
#fnc = "initial_positions_bottom_ORCA12.txt"
fnc = "initial_positions.txt"

text_file = open(fnc, "w")

count = 0

for jj in range(1,ny-1):
    for ii in range(1,nx-1):
        if tmasksurf[jj,ii]> 0.5:
            text_file.write(str(float(ii+0.5))  + "  " +
                            str(float(jj+0.5))  + "  " +
                            str(float(1.5))     + "  " +
                            str(float(0.5))     + "  " +
                            str(1.0)            + "\n" )
            count += 1
            
    
text_file.close()

print("number of initial positions =",count)

## Initialisation: 2.10$^7$ particles (ORCA025)

### Air of the ocean

In [7]:
# Ocean area 
A_oce = e1t * e2t * tmasksurf
A_oce = np.nansum(A_oce)
print(f'Total area of ocean: {A_oce:.2e}m²')

Total area of ocean: 3.61e+14m²


### Set particles in mech with concentration = 1 

In [8]:
# Truncation function
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [ ]:
# Create initial_positions.txt file and fill it with concentration to 1 

fnc = "initial_positions_C1.txt"

text_file = open(fnc, "w")

eps = 0.001           # Ariane can't deal if particle are on the bunderies: distance from the bounderies 
n_tot = 2e7           # Number of total particles put in the ocean 
alpha = n_tot/A_oce   # Concentration of particles in the ocean 

for jj in range(1,ny-1):
    for ii in range(1,nx-1):
        Aii_jj = e1t[jj,ii] * e2t[jj,ii] * tmasksurf[jj,ii]
        nii_jj = int(alpha * Aii_jj) # Number of particles per mesh 
        if nii_jj > 0.5:
            for nn in range(nii_jj):
            # Get ramdom particles between ii+eps and ii+1-eps
                text_file.write(str(truncate(random.uniform(ii+eps, ii+1-eps), 3))  + "  " +
                                str(truncate(random.uniform(jj+eps, jj+1-eps), 3))  + "  " + 
                                str(float(1.5))     + "  " + # Initial time
                                str(float(0.5))     + "  " + # On surface 
                                str(1.0)            + "\n" )
        #count += nii_jj
            
    
text_file.close()

## Initialisation: 2.10$^3$ particles per zones (ORCA025)

In [36]:
# Define slices for the zones
zones = [
    (slice(949, 952), slice(975, 978)),  # Zone 1
    (slice(899, 902), slice(975, 978)),  # Zone 2 
    (slice(799, 802), slice(975, 978)),  # Zone 3 
]

fnc = "initial_positions_zones.txt"

text_file = open(fnc, "w")

eps = 0.001     # Ariane can't deal if particle are on the bunderies: distance from the bounderies 
n_zones = 6e3   # Number of particles put in the 3 zones 
  
A_zones = (e1t[zones[0]]*e2t[zones[0]]*tmasksurf[zones[0]])+ \
        (e1t[zones[1]]*e2t[zones[1]]*tmasksurf[zones[1]])  + \
        (e1t[zones[2]]*e2t[zones[2]]*tmasksurf[zones[2]])
        
alpha = n_zones / A_zones.sum() 
           
for ilat, ilon in zones:
    lon = np.arange(ilon.start, ilon.stop)
    lat = np.arange(ilat.start, ilat.stop)
    for jj in lat:
        for ii in lon:
            Aii_jj = e1t[jj,ii] * e2t[jj,ii] * tmasksurf[jj,ii]
            nii_jj = int(alpha * Aii_jj) # Number of particles per mesh 
            if nii_jj > 0.5:
                for nn in range(nii_jj):
                # Get ramdom particles between ii+eps and ii+1-eps
                    text_file.write(str(truncate(random.uniform(ii+eps, ii+1-eps), 3))  + "  " +
                                    str(truncate(random.uniform(jj+eps, jj+1-eps), 3))  + "  " + 
                                    str(float(1.5))     + "  " + # Initial time
                                    str(float(0.5))     + "  " + # On surface 
                                    str(1.0)            + "\n" )

text_file.close()
          